In [19]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import folium
from folium.features import DivIcon

부산 버스 모든 노선의 ID를 구해서 xlsx파일로 저장하기.

In [2]:
bus_all_line = []
def busan_bus_line():
    url = 'http://apis.data.go.kr/6260000/BusanBIMS/busInfo'
    params ={'serviceKey' : 'l2SER5UdFnVMTCfW/75Op1w1xj0ZVJyWjxZCA4s2qcMkJIjXonsQeorj8k35mmylOV0ywqc75RoXXtagwXuUnQ=='}

    response = requests.get(url, params=params)
    decodecontent = response.content.decode('utf-8')
    xmlParser = BeautifulSoup(decodecontent,'xml')
    contents =  xmlParser.select('item')
    for number in range(0,len(contents)):
        content = contents[number].text.split('\n')
        content.remove('')
        content.remove('')
        bus_all_line.append(content)


busan_bus_line()


In [18]:
df_bus_line.columns = ['노선ID','버스 번호','버스 종류','기점명','종점명','버스 회사명','배차간격','첫차','막차','배차간격(출퇴근)',
                       '배차간격(평일)','배차간격(휴일)']

In [19]:
df_bus_line.to_excel('./data/부산버스라인.xlsx',index=False)

부산의 버스 라인으로 검색하면 노선의 ID로 바꾸어 검색하기

In [4]:
bus_id_num = pd.read_excel('./data/부산버스라인.xlsx',usecols=['노선ID','버스 번호'])

In [10]:
# 버스 번호에 해당하는 노선 ID 찾기
def bus(num):
    condition = bus_id_num['버스 번호'] == str(num)
    index = bus_id_num[condition].index
    return bus_id_num.values[index][0][0]
    

print(bus('1002(심야)'))

5201002F00


In [44]:
exam = bus(1002)
exam

'5201002000'

<?xml version="1.0" encoding="utf-8"?>
<response>
<header>
<resultCode>00</resultCode>
<resultMsg>NORMAL SERVICE</resultMsg>
</header>
<body>
<items>
<item>
<bstopidx>1</bstopidx>
<bstopnm>금정공영차고지</bstopnm>
<nodeid>217210101</nodeid>
<lineno>80</lineno>
<rpoint>0</rpoint>
</item>
<item>
<bstopidx>2</bstopidx>
<bstopnm>부산종합터미널.노포역</bstopnm>
<nodeid>209010201</nodeid>
<lineno>80</lineno>
<arsno>11006</arsno>
<avgym>180</avgym>
<rpoint>0</rpoint>
</item>
<item>
<bstopidx>3</bstopidx>
<bstopnm>노포삼거리</bstopnm>
<nodeid>210760301</nodeid>
<lineno>80</lineno>
<direction>1</direction>
<gpsym>212434</gpsym>
<carno>70자3157</carno>
<lat>35.2798816</lat>
<lin>129.0932608</lin>
<nodekn>0</nodekn>
<arsno>11009</arsno>
<avgym>87</avgym>
<rpoint>0</rpoint>
<lowplate>0</lowplate>
</item>
<item>
<bstopidx>4</bstopidx>
<bstopnm>범어사입구</bstopnm>
<nodeid>175290301</nodeid>
<lineno>80</lineno>
<arsno>11012</arsno>
<avgym>142</avgym>
<rpoint>0</rpoint>
</item>
<item>
<bstopidx>5</bstopidx>
<bstopnm>어린이놀이터</bst

In [45]:
bus_now_list= []
def bus_now(bus_num):
    condition = bus_id_num['버스 번호'] == str(bus_num)
    index = bus_id_num[condition].index
    lineid = bus_id_num.values[index][0][0]
    url = 'http://apis.data.go.kr/6260000/BusanBIMS/busInfoByRouteId'
    params ={'serviceKey' : 'l2SER5UdFnVMTCfW/75Op1w1xj0ZVJyWjxZCA4s2qcMkJIjXonsQeorj8k35mmylOV0ywqc75RoXXtagwXuUnQ==',
             'lineid' : lineid}
    response = requests.get(url, params=params)
    decodecontent = response.content.decode('utf-8')
    xmlParser = BeautifulSoup(decodecontent,'xml')
    contents = xmlParser.select('item')
    for number in range(0,len(contents)):
        content = contents[number].text.split('\n')
        content.remove('')
        content.remove('')
        bus_now_list.append(content)
    return bus_now_list

In [49]:
exam = bus_now(1002)

In [6]:
stop_general = pd.read_excel('./data/부산버스_일반.xlsx')
location = stop_general[['GPS X 좌표','GPS Y 좌표']]

In [ ]:
map =folium.Map(location=[location['GPS Y 좌표'].mean(),location['GPS X 좌표'].mean()],zoom_start=12)


for i in range(0,len(exam)):
    try:
        lat = exam[i][7]
        lng = exam[i][8]
        bus_num = exam[i][3]
        if exam[i][4] == '1':
            folium.CircleMarker(location=[lat,lng],
                        fill=True,fill_color='red',fill_opacity=2,color='yellow',radius=5,weight=1).add_to(map)
            folium.Marker(location=[lat,lng],icon=DivIcon(
                icon_size=(100,60),
                icon_anchor=(75,25),
                html = f'''
                <div style="font-size : 9pt; font-weight: bold ; color:black; text-align:center;">
                {exam[i][3]}번 버스
                </div>
                '''
            )).add_to(map)
        else:
            folium.CircleMarker(location=[lat,lng],
                fill=True,fill_color='blue',fill_opacity=2,color='yellow',radius=5,weight=1).add_to(map)
            folium.Marker(location=[lat,lng],icon=DivIcon(
                icon_size=(100,60),
                icon_anchor=(75,25),
                html = f'''
                <div style="font-size : 9pt; font-weight: bold ; color:black; text-align:center;">
                {exam[i][3]}번 버스
                </div>
                '''
            )).add_to(map)
    except:
        pass

In [60]:
## 함수로 선언하기

stop_general = pd.read_excel('./data/부산버스_일반.xlsx')
location = stop_general[['GPS X 좌표','GPS Y 좌표']]
bus_now_list= []


map =folium.Map(location=[location['GPS Y 좌표'].mean(),location['GPS X 좌표'].mean()],zoom_start=12)


def bus_now(bus_num):
    condition = bus_id_num['버스 번호'] == str(bus_num)
    index = bus_id_num[condition].index
    lineid = bus_id_num.values[index][0][0]
    url = 'http://apis.data.go.kr/6260000/BusanBIMS/busInfoByRouteId'
    params ={'serviceKey' : 'l2SER5UdFnVMTCfW/75Op1w1xj0ZVJyWjxZCA4s2qcMkJIjXonsQeorj8k35mmylOV0ywqc75RoXXtagwXuUnQ==',
            'lineid' : lineid}
    response = requests.get(url, params=params)
    decodecontent = response.content.decode('utf-8')
    xmlParser = BeautifulSoup(decodecontent,'xml')
    contents = xmlParser.select('item')
    for number in range(0,len(contents)):
        content = contents[number].text.split('\n')
        content.remove('')
        content.remove('')
        bus_now_list.append(content)
        for i in range(0,len(bus_now_list)):
            try:
                lat = bus_now_list[i][7]
                lng = bus_now_list[i][8]
                bus_num = bus_now_list[i][3]
                if bus_now_list[i][4] == '1':
                    folium.CircleMarker(location=[lat,lng],
                                fill=True,fill_color='red',fill_opacity=2,color='yellow',radius=5,weight=1).add_to(map)
                    folium.Marker(location=[lat,lng],icon=DivIcon(
                        icon_size=(100,60),
                        icon_anchor=(75,25),
                        html = f'''
                        <div style="font-size : 9pt; font-weight: bold ; color:black; text-align:center;">
                        {bus_now_list[i][3]}번 버스
                        </div>
                        '''
                    )).add_to(map)
                elif bus_now_list[i][4] == '2':
                    folium.CircleMarker(location=[lat,lng],
                        fill=True,fill_color='blue',fill_opacity=2,color='yellow',radius=5,weight=1).add_to(map)
                    folium.Marker(location=[lat,lng],icon=DivIcon(
                        icon_size=(100,60),
                        icon_anchor=(75,25),
                        html = f'''
                        <div style="font-size : 9pt; font-weight: bold ; color:black; text-align:center;">
                        {bus_now_list[i][3]}번 버스
                        </div>
                        '''
                    )).add_to(map)
            except:
                pass
    map.save('./data/부산버스 현재 위치.html')

In [59]:
bus_now(131)